In [1]:
!nvidia-smi

Tue May  6 20:51:53 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GH200 120GB             On  | 00000009:01:00.0 Off |                    0 |
| N/A   26C    P0              92W / 900W |   5413MiB / 97871MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import json

import torch
import transformers
from environs import env
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from local_funcs import prompt_funcs
from yiutils.project_utils import find_project_root

proj_root = find_project_root("justfile")
data_dir = proj_root / "data"

print(transformers.__version__)
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

env.read_env(proj_root / ".env")
access_token = env("HUGGINGFACE_TOKEN")

path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists(), (
    f"Data file {path_to_mr_pubmed_data} does not exist."
)

with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)

article_data = mr_pubmed_data[0]

message_metadata = prompt_funcs.make_message_metadata(article_data["ab"])
message_results = prompt_funcs.make_message_results(article_data["ab"])


/home/b5a/ik18445.b5a/micromamba/envs/data-extraction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3
2.6.0
True
12.6


In [3]:
MODEL_ID = "tiiuae/falcon-40b-instruct"

device = "auto"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map=device,
    token=access_token,
    # trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 9/9 [00:37<00:00,  4.19s/it]


In [4]:
messages = message_metadata
# messages = message_results
input_ids = tokenizer.apply_chat_template(
    conversation=messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)
print(input_ids.shape)
input_ids

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [10]:
terminators = [
    tokenizer.eos_token_id,
    # tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]
outputs = model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    # top_p=0.95,
)
print(outputs.shape)
outputs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


torch.Size([1, 1561])


tensor([[1357,  362,  241,  ..., 1881,   25,   11]], device='cuda:0')

In [11]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'You are a data scientist responsible for extracting accurate information from research papers. You answer each question with a single JSON string.\n\nUser: This is an abstract from a Mendelian randomization study.\n                    "Alcohol consumption significantly impacts disease burden and has been linked to various diseases in observational studies. However, comprehensive meta-analyses using Mendelian randomization (MR) to examine drinking patterns are limited. We aimed to evaluate the health risks of alcohol use by integrating findings from MR studies. A thorough search was conducted for MR studies focused on alcohol exposure. We utilized two sets of instrumental variables-alcohol consumption and problematic alcohol use-and summary statistics from the FinnGen consortium R9 release to perform de novo MR analyses. Our meta-analysis encompassed 64 published and 151 de novo MR analyses across 76 distinct primary outcomes. Results show that a genetic predisposition to alcohol consu

In [12]:
response = outputs[0][input_ids.shape[-1] :]
result = tokenizer.decode(response, skip_special_tokens=True)
print(result)

 This is an example output in JSON format: 
    {
    "results": [
        {
            "exposure": "Particulate matter 2.5"},
            "outcome": "Forced expiratory volume in 1 s"
            "beta": 0.154,
            "units": "mmHg",
            "hazard ratio": null,
            "odds ratio": null,
            "95% CI": [0.101,0.215],
            "SE": 0.102,
            "P-value": 0.0015,
            "direction": "increases"
        },
        {
            "exposure": "Body mass index"},
            "outcome": "Gastroesophageal reflux disease"
            "beta": null,
            "units": null,
            "hazard ratio": null,
            "odds ratio": 1.114,
            "95% CI": [1.021,1.314],
            "SE": null,
            "P-value": 0.0157,
            "direction": "increases"
        },
        {
            "exposure": "Body mass index"},
            "outcome": "Non-alcoholic fatty liver disease (NAFLD)"
            "beta": null,
            "units": null,
       